In [1]:
import requests, re, urllib, itertools
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse
from collections import Counter, Iterable
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
from anytree import Node
from nltk.tree import Tree
from nltk.chunk.regexp import RegexpParser
from nltk import pos_tag, word_tokenize

def checkAppropriateURL(url):
    parsedURL = urlparse(url)   

    extn = parsedURL.path.split(".")[-1]
    site = parsedURL.netloc.split(".")
    path = parsedURL.path.split("/")[-1]

    if extn in ["pdf","doc","docx","xls","xlsx","ppt","pptx","odt"]:
        return False
    if any(i in site for i in ["pinterest"]):
        return False
    r = dict(requests.head(url).headers)["Content-Type"].split(";")[0]
    if(not r=="text/html"):
        return False
    return True

def scrapeGoogleForAbstracts(query, count):
    words = query.split()
    q = ("+").join(words)
    print (q)
    query_url_goog = "https://www.google.co.in/search?q=\"" + q + "\""
    data = requests.get(query_url_goog).text
    print (data)
    soup = BeautifulSoup(data, "lxml")
    h3Rows = soup.find_all("h3", {"class":"r"})
    spanRows = soup.find_all("span", {"class":"st"})
    zippedRows = zip(h3Rows,spanRows)
    urls = []
    print (list(zippedRows))
    for zippedRow in zippedRows:
        try:
            h3Row = zippedRow[0]
            url = h3Row.find("a")['href']
            if(url.split("?")[0]=="/search"):
                continue
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        par = urllib.parse.parse_qs(urlparse(url).query)
        try:
            appendingURL = par['q'][0]
        except:
            continue
        if(not checkAppropriateURL(appendingURL)):
            continue
        matchedQuery = query
        allbElems = zippedRow[1].findAll('b')
        for bElem in allbElems:
            if bElem.string[0].isalnum():
                matchedQuery = bElem.string
        urls.append((matchedQuery,appendingURL))
    i = 1
    same = 0
    prev = 0
    while(len(urls)<count):
#         print ("Query: ",query)
        tempurl = query_url_goog + "&start=" +str(i*10)
        i+=1
        data = requests.get(tempurl).text
        soup = BeautifulSoup(data, "lxml")
        h3Rows = soup.find_all("h3", {"class":"r"})
        spanRows = soup.find_all("span", {"class":"st"})
        zippedRows = zip(h3Rows,spanRows)
        for (j,zippedRow) in enumerate(zippedRows):
            try:
                h3Row = zippedRow[0]
                url = h3Row.find("a")['href']
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
            par = urllib.parse.parse_qs(urlparse(url).query)
            try:
                appendingURL = par['q'][0]
                if(not checkAppropriateURL(appendingURL)):
                    continue
                matchedQuery = query
                allbElems = zippedRow[1].findAll('b')
                for bElem in allbElems:
                    if bElem.string[0].isalnum():
                        matchedQuery = bElem.string
                urls.append((matchedQuery, appendingURL))
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
        if prev == len(urls):
            same +=1 
            if same>=10:
                same = 0
                break
        prev = len(urls)
    return urls[:count]

In [3]:
from gensim.models import KeyedVectors

text = open("test.txt").read()
model = KeyedVectors.load_word2vec_format("~/GoogleNews-vectors-negative300.bin", binary=True)
model["security"]

array([ 0.0189209 ,  0.18554688,  0.04394531, -0.0703125 , -0.04125977,
       -0.03515625, -0.08251953, -0.10351562,  0.0625    ,  0.04931641,
        0.04882812, -0.28710938,  0.00585938,  0.01660156, -0.24609375,
        0.01275635, -0.08251953,  0.03295898, -0.11816406, -0.24316406,
       -0.12304688, -0.13476562,  0.0703125 ,  0.3046875 , -0.01696777,
       -0.01916504, -0.01733398, -0.12695312,  0.07666016, -0.24511719,
        0.13183594, -0.171875  , -0.00753784,  0.00750732, -0.10693359,
       -0.49609375, -0.07226562,  0.01733398,  0.01318359,  0.15820312,
       -0.22070312,  0.04248047,  0.11816406,  0.06005859, -0.09863281,
       -0.03686523, -0.01116943, -0.05273438, -0.13476562,  0.07958984,
       -0.00305176,  0.22753906, -0.08007812,  0.08642578, -0.23144531,
        0.14453125, -0.12988281,  0.13867188, -0.13867188, -0.38476562,
       -0.26953125,  0.13867188, -0.03222656,  0.05053711,  0.02038574,
        0.00300598, -0.078125  ,  0.04248047,  0.06176758, -0.13

In [2]:


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def getTextFromURL(url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
    html = urllib.request.urlopen(urllib.request.Request(url, headers=hdr))
    inputText = text_from_html(html)
#     print (inputText)
    return inputText

def getAbstracts(query, count):
    urls = scrapeGoogleForAbstracts(query, count)
    print (urls)
    print ("lenght",len(urls))
    texts = []
    for (query, url) in urls:
        print (url)
        try:
            text = getTextFromURL(url)
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        texts.append((query,url,text.strip()))
    return texts


In [3]:
def processWordSet(wordSet):
    for (i,word) in enumerate(wordSet):
        wordSet[i] = re.sub("^[^a-zA-Z]*|[^a-zA-Z(?<!(s\'))]*$","",word)
    return wordSet

def getHypernymsFromWordnet(word):
    allSynsets = wn.synsets(word)
    multiList = [[[a.name() for a in b.lemmas()] for b in c.hypernyms()]for c in wn.synsets(word)]
    finalList = flatten(multiList)
    return Counter(finalList)

def compareTuple(x,model):
    try:
        sim = model.similarity(x[0],x[1])
        return sim
    except:
        return 0

def initNodesList(wordSet):
    nodesList = []
    for wordNode in wordSet:
        nodesList.append(Node(wordNode))
    return nodesList

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

# file = open("pizza.txt")
# text = file.read()
# wordSet = processWordSet(list(set(text.split())))
# wordSet = list(set(filter(None, wordSet)))

wordSet = ["vulnerability","threat"]

model = KeyedVectors.load_word2vec_format("~/GoogleNews-vectors-negative300.bin", binary=True)

# Sort by decreasing similarity
tuplesList = list(itertools.combinations(wordSet,2))
sortedList = sorted(tuplesList, key=lambda x : compareTuple(x,model), reverse=True)

clusteredSet = set()
hypernymDict = {}

root = Node("root")
nodesList = initNodesList(wordSet)


/usr/local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [4]:
def getHearstPatterns(word):
    # Pattern, Instances after/before, One/many instances
    patterns = []
    patterns.append((word + " such as", True))
    patterns.append(("such " + word + " as", True))
    patterns.append(("and other " + word, False))
    patterns.append(("or other " + word, False))
    patterns.append((word + ", including", True))
    patterns.append((word + ", especially", True))
    return patterns

def getInstances(text):
    grammar = """
        PRE:   {<NNS|NNP|NN|NP|JJ|UH>+}
        MID: {<DT|IN|POS|FW|-|NP|NPS|NN|NNS>+}
        INSTANCE:   {(<DT+>)?(<JJ+>)?<PRE>(<MID><PRE>)?}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "INSTANCE"):
            # print (i.leaves())
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

def extractHypernyms(posList, text, query):
    allInstances = []
    for (st,end) in posList:
        if query[1]:
            #Instances after, and exists                
            tempText = text[end+1:]
            rest = re.match(".*?\.",tempText)
            if rest:
                rest = rest.group()
            else:
                rest = tempText
            andExists = re.search(" (and|or) ", rest)
            if andExists:
                print ("rest:", rest)
                lastConceptPos = andExists.end()
                lastConcept = rest[lastConceptPos:]
                print ("lastConcept: ", lastConcept)
                lastConceptExtracted = getInstances(lastConcept.strip())
                if lastConceptExtracted:
                    allInstances.append(lastConceptExtracted[0])
                prevConceptsEnd = andExists.start()
                prevConcepts = rest[:prevConceptsEnd]
                prevConceptsList = prevConcepts.split(",")
                prevConceptsList = [a for a in prevConceptsList if a.strip()]
                print ("****START")
                print ("previous Concepts:",prevConcepts)
                for concept in prevConceptsList:
                        print (concept.strip(), getInstances(concept.strip()))
                        conceptExtracted = getInstances(concept.strip())
                        if conceptExtracted:
                            allInstances.append(conceptExtracted[-1])
                print ("*****DONE")
            else:
                allConcepts = rest.split(",")
                print ("#####HERE ARE ALL CONCEPTS:",allConcepts)
                for (i,concept) in enumerate(allConcepts):
                        if i==(len(allConcepts)-1):
                            print (concept)
                            conceptExtracted = getInstances(concept.strip())
                            print ("1######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[0])
                        else:
                            conceptExtracted = getInstances(concept.strip())
                            print ("######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[-1])
        else:
            tempText = text[:st]
            print ("tempText", tempText)
            try:
                lastSentPos = [m.end() for m in re.finditer(".*?\.",tempText)][-1]
                lastSent = tempText[:lastSentPos]
            except:
                lastSent = tempText
            lastSentList = lastSent.split(",")
            lastSentList = [a for a in lastSentList if a.strip()]
            if len(lastSentList) == 1:
                conceptExtracted = getInstances(lastSentList[0].strip())
                if conceptExtracted:
                    allInstances.append(conceptExtracted[-1])
            else:
                for (i,concept) in enumerate(lastSentList):
                    conceptExtracted = getInstances(concept.strip())
                    if conceptExtracted:
                        if i==(len(lastSentList)-1):
                            allInstances.append(conceptExtracted[0])
                        else:
                            allInstances.append(conceptExtracted[-1])
    return allInstances

def getHypernymsFromWWW(word):
    allHypernyms = []
    queriesList = getHearstPatterns(word)
    for query in queriesList:
        texts = getAbstracts(query[0], 10)
        for (q,url,text) in texts:
            startEndPositions = []
            try:
                for m in re.finditer(re.escape(q), text):
                    try:
                        startEndPositions.append((m.start(), m.end()))
                    except:
                        print ("query:",q,"url:",url)
                        continue
            except:
                print ("1","query:",q,"url:",url)
                continue
            allHypernyms.extend(extractHypernyms(startEndPositions, text, query))
    return Counter(allHypernyms)

def getHypernymsFromTextCorpus(word):
    allInstances = []
    for quer in ["such as", "including", "especially", "such_as"]:
        if quer == "such_as":
            regex = "such " + word + " as" + ".*?\." 
        else:
            regex = word + ".*?\."
        allMatches = re.findall(regex, textCorpus)
        for match in allMatches:
            if quer=="such_as":
                rest = match
            else:
                rest = re.search(quer+".*", match)
                if rest:
                    rest = rest.group()
                else:
                    continue
            andExists = re.search(" (and|or) ", rest)
            if andExists:
                print ("rest:", rest)
                lastConceptPos = andExists.end()
                lastConcept = rest[lastConceptPos:]
                print ("lastConcept: ", lastConcept)
                lastConceptExtracted = getInstances(lastConcept.strip())
                if lastConceptExtracted:
                    allInstances.append(lastConceptExtracted[0])
                prevConceptsEnd = andExists.start()
                prevConcepts = rest[:prevConceptsEnd]
                prevConceptsList = prevConcepts.split(",")
                prevConceptsList = [a for a in prevConceptsList if a.strip()]
                print ("****START")
                print ("previous Concepts:",prevConcepts)
                for concept in prevConceptsList:
                        print (concept.strip(), getInstances(concept.strip()))
                        conceptExtracted = getInstances(concept.strip())
                        if conceptExtracted:
                            allInstances.append(conceptExtracted[-1])
                print ("*****DONE")
            else:
                allConcepts = rest.split(",")
                print ("#####HERE ARE ALL CONCEPTS:",allConcepts)
                for (i,concept) in enumerate(allConcepts):
                        if i==(len(allConcepts)-1):
                            print (concept)
                            conceptExtracted = getInstances(concept.strip())
                            print ("1######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[0])
                        else:
                            conceptExtracted = getInstances(concept.strip())
                            print ("######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[-1])
        
    for quer in ["and other ", "or other "]:
        regex = "\..*?" + word 
        allMatches = re.findall(regex, textCorpus)
        for match in allMatches:
            rest = re.search(".*" + quer, match)
            if rest:
                rest= rest.group()
            else:
                continue
            allConcepts = rest.split(",")
            print ("#####HERE ARE ALL CONCEPTS:",allConcepts)
            for (i,concept) in enumerate(allConcepts):
                conceptExtracted = getInstances(concept.strip())
                print ("######CONCEPT EXTRACTED:", conceptExtracted)
                if conceptExtracted:
                    allInstances.append(conceptExtracted[-1])
    return Counter(allInstances)

def getHypernyms(word):
    wordnet = getHypernymsFromWordnet(word)
    www = getHypernymsFromWWW(word)
    text = getHypernymsFromTextCorpus(word)
    return (wordnet + www + text)
        

In [5]:
import textract
textCorpus = ""
import os

directory = os.fsencode("SecurityPDFs")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".pdf"): 
        textCorpus += textract.process("SecurityPDFs/" + filename).decode("utf-8")
        continue


In [9]:
wordSet = ["vulnerability","threat"]
hypernymDict = {}
for word in wordSet:
    hypernymDict[word] = getHypernyms(word)
    

vulnerability+such+as
<!doctype html><html itemscope="" itemtype="http://schema.org/SearchResultsPage" lang="en-IN"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><noscript><meta content="0;url=/search?q=%22vulnerability+such+as%22&amp;ie=UTF-8&amp;gbv=1&amp;sei=qH6XXMWkDv3dz7sPgeCFoAg" http-equiv="refresh"><style>table,div,span,p{display:none}</style><div style="display:block">Please click <a href="/search?q=%22vulnerability+such+as%22&amp;ie=UTF-8&amp;gbv=1&amp;sei=qH6XXMWkDv3dz7sPgeCFoAg">here</a> if you are not redirected within a few seconds.</div></noscript><title>"vulnerability such as" - Google Search</title><style>#gbar,#guser{font-size:13px;padding-top:1px !important;}#gbar{height:22px}#guser{padding-bottom:7px !important;text-align:right}.gbh,.gbd{border-top:1px solid #c9d7f1;font-size:1px}.gbh{height:0;position:absolute;top:24px;width:100%}@media all{.gb1{

[('vulnerability, such as', 'https://www.nap.edu/read/2050/chapter/2'), ('vulnerability, such as', 'https://portswigger.net/kb/issues/00400200_password-returned-in-later-response'), ('vulnerability, such as', 'http://www.scielo.br/scielo.php?script=sci_arttext&pid=S1414-753X2018000100403'), ('vulnerability such as', 'https://research.checkpoint.com/extracting-code-execution-from-winrar/'), ('vulnerability, such as', 'https://cwe.mitre.org/about/faq.html'), ('as', 'https://ieeexplore.ieee.org/document/889093'), ('vulnerability, such as', 'https://www.microsoft.com/en-us/msrc/bounty-example-report-submission'), ('vulnerability, such as', 'https://www.preventionweb.net/go/62020'), ('vulnerability, such as', 'https://journals.sagepub.com/doi/full/10.1177/1070496516682339'), ('vulnerability such as', 'https://publications.iom.int/books/mongolia-urban-migrant-vulnerability-assessment')]
lenght 10
https://www.nap.edu/read/2050/chapter/2
https://portswigger.net/kb/issues/00400200_password-retu

rest:  History  Search History  What can I access?     Get Help   Feedback  Technical Support  Resources and Help                        IEEE Account   Change Username/Password  Update Address     Purchase Details   Payment Options  Order History  View Purchased Documents     Profile Information   Communications Preferences  Profession and Education  Technical Interests     Need Help?   US & Canada: +1 800 678 4333  Worldwide: +1 732 981 0060   Contact & Support          About IEEE Xplore  Contact Us  Help  Accessibility  Terms of Use  Nondiscrimination Policy  Sitemap  Privacy & Opting Out of Cookies   A not-for-profit organization, IEEE is the world's largest technical professional organization dedicated to advancing technology for the benefit of humanity.
lastConcept:  Help                        IEEE Account   Change Username/Password  Update Address     Purchase Details   Payment Options  Order History  View Purchased Documents     Profile Information   Communications Preferences 

[('vulnerability. As', 'https://www.ssrn.com/abstract=2391112'), ('such vulnerability as', 'https://www.sotosllp.com/2000/01/the-fiduciary-relationship-in-franchising-jirna-reconsidered/2/'), ('such vulnerability as', 'https://news.unocoin.com/?paged=6&cat=2'), ('vulnerability. As', 'https://news.unocoin.com/?paged=16'), ('such vulnerability, as', 'https://www.karger.com/Article/Pdf/330039'), ('such vulnerability as', 'https://qz.com/work/1454030/the-easiest-and-most-effective-way-to-make-your-colleagues-feel-happy-and-safe-at-work/'), ('such vulnerability. As', 'https://www.tandfonline.com/doi/full/10.1080/10615806.2013.853047?src=recsys'), ('such vulnerability? As', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4888812/'), ('such vulnerability as', 'http://iopscience.iop.org/article/10.1088/1755-1315/145/1/012142/pdf'), ('such vulnerability, as', 'http://www.tnsglobal.com/press-release/embracing-crowdsourcing-right-reasons')]
lenght 10
https://www.ssrn.com/abstract=2391112
https://ww

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

tempText US20070186284A1 - Geographical Threat Response Prioritization Mapping System And Methods Of Use 
        - Google Patents  Geographical Threat Response Prioritization Mapping System And Methods Of Use   Download PDF  Info   Publication number  US20070186284A1     US20070186284A1  US11/617,152  US61715206A  US2007186284A1  US 20070186284 A1  US20070186284 A1  US 20070186284A1        US 61715206 A  US61715206 A  US 61715206A  US 2007186284 A1  US2007186284 A1  US 2007186284A1  Authority  US  United States  Prior art keywords  threat  information  location  database  data  Prior art date  2004-08-12  Legal status (The legal status is an assumption and is not a legal conclusion. Google has not performed a legal analysis and makes no representation as to the accuracy of the status listed.)   Granted    Application number  US11/617,152  Other versions    US8418246B2 ( en    Inventor  James McConnell  Current Assignee (The listed assignees may be inaccurate. Google has not performed 

tempText Forensic linguistics   From Wikipedia, the free encyclopedia    Jump to navigation  Jump to search    Part of a series on Forensic science  Physiological  Anthropology  Biology  Bloodstain pattern analysis  Botany  Dentistry  DNA phenotyping  DNA profiling  Entomology  Epidemiology  Medicine  Palynology  Pathology  Podiatry  Toxicology    Social  Psychiatry  Psychology  Psychotherapy  Social work   Criminalistics  Accounting  Body identification  Chemistry  Facial reconstruction  Fingerprint analysis  Firearm examination  Footwear evidence  Forensic arts  Profiling  Gloveprint analysis  Palmprint analysis  Questioned document examination  Vein matching  Forensic geophysics  Forensic geology    Digital forensics  Computer exams  Data analysis  Database study  Malware analysis  Mobile devices  Network analysis  Photography  Video analysis  Audio analysis    Related disciplines  Electrical engineering  Engineering  Fire investigation  Fire accelerant detection  Fractography  Ling

tempText New South Wales Consolidated Acts [ Index ]
[ Table ]
[ Search ]
[ Search this Act ]
[ Notes ]
[ Noteup ]
[ Previous ]
[ Next ]
[ Download ]
[ Help ]  CRIMES ACT 1900 - SECT 80A  Sexual assault by forced self-manipulation  80A Sexual assault by forced self-manipulation   (1) In this section:  "circumstances of aggravation" means circumstances in which: (a) at the time
of, or immediately before or after, the commission of the offence, the alleged
offender intentionally or recklessly inflicts actual bodily harm on the alleged victim or any other person who is present or nearby, or  (b) at the
time of, or immediately before or after, the commission of the offence, the
alleged offender threatens to inflict actual bodily harm on the alleged victim or any other person who is present or nearby by means of an offensive weapon or instrument , or  (c) the alleged offender is in the company
of another person or persons , or  (d) the alleged victim is under the age of
16 years, or  (e) th

[('threat, including', 'http://www.cvent.com/events/cybersecurity-confronting-the-threat-sharing-information-and-spreading-the-risk-virtual-class/event-summary-a7872d7df0e343cebf2c772a12f8beb5.aspx'), ('threat, including', 'https://securelist.com/threats-posed-by-using-rats-in-ics/88011/'), ('threat, including', 'https://www.publicsafety.gc.ca/cnt/rsrcs/pblctns/pblc-rprt-trrrsm-thrt-cnd-2018/index-en.aspx'), ('threat, including', 'https://bluehexagon.ai/press/blue-hexagon-announces-integration-with-carbon-black-to-set-new-standard-for-accelerated-threat-prevention/'), ('threat, including', 'https://www.mcafee.com/enterprise/en-in/threat-center/advanced-threat-research.html'), ('including', 'https://www.metricstream.com/factsheets/stvm-app.htm'), ('threat, including', 'https://apmg-international.com/product/gct/insider-threat-including-social-media-best-practice'), ('threat, including', 'https://www.computerworld.com/article/2516690/cyberattacks-an--existential-threat--to-u-s---fbi-says

[('threat, especially', 'https://www.researchgate.net/figure/Daily-activities-like-cooking-and-heating-pose-a-threat-especially-to-women-and-children_fig1_331043616'), ('threat, especially', 'https://www.washingtonexaminer.com/isis-is-still-a-threat-especially-to-the-uk'), ('especially', 'https://www.ssrn.com/abstract=3311466'), ('threat, especially', 'https://www.zixcorp.com/resources/blog/august-2018/extortion-emails-old-threat-new-twist'), ('threat, especially', 'https://www.ust-global.com/cyber-global-asset-management'), ('threat; especially', 'https://www.beyondtrust.com/blog/entry/when-a-spear-phisher-came-for-me'), ('threat especially', 'https://blogs.technet.microsoft.com/educloud/2017/06'), ('threat, especially', 'https://www.dw.com/en/nato-flexes-muscles-in-poland-against-russian-threat/av-47865501'), ('threat (especially', 'https://www.emeraldinsight.com/doi/10.1108/S0163-786X%282012%290000033012'), ('threat, especially', 'https://www.ncbi.nlm.nih.gov/pubmed/30363509')]
leng

In [19]:
# finalDict = {}
# for hypernym in hypernymDict:
#     finalDict[hypernym] = {}
#     for word in hypernymDict[hypernym]:
#         if hypernymDict[hypernym][word] > 1:
#             finalDict[hypernym][word] = hypernymDict[hypernym][word]
# finalDict

finalStr = ""
for hypernym in hypernymDict:
    counter = hypernymDict[hypernym]
    finalStr += "Hyponyms of " + hypernym + ":\n\n" + str(len(counter.keys())) + " concepts" + "\n\n" + "\n".join(counter.keys()) + "\n\n\n\n"

open("Clustering_Results.txt","w+").write(finalStr)

61449

In [16]:
"\n".join(hypernymDict['threat'].keys())

"danger\nwarning\ndeclaration\nperson\nindividual\nsomeone\nsomebody\nmortal\nsoul\nlimited distribution Enhanced analysis\nsecurity vendor report\nOpen Source intelligence\nreset instructions Email Cancel If the address\nemail address\nusername Email Close If the address\ns\nDOI\nkeywords\nauthors\netc\nFellows Events Grants\nschemes\nstudent\nFERPA\nschool official\nother articles\nintroduce links\nsociology\ntheory in psychology\neffect\nlevels of prejudice\nWalter G\npart of realistic conflict theory\n’ s identity\n’ s values\ncolleagues\nEsses\nmembers of the outgroup\ned\nunsafe\ngeneral stereotypes\nlinks between prejudice\nother groups\nnegative aspects of stereotypes\nects of stereotypes\nsymbolic threats\nrealistic\nc framework of the theory\nsubtypes of threat\nown separate categories\nphysical danger\nrealistic threat\nsymbolic threat\nthreat in order\ntypes of realistic\nsuccess\nlikely\nfavorable attitudes towards immigrants\nsupportive of programs\nnegative attitudes tow